In [1]:
from ast import literal_eval
from collections import defaultdict

import pandas as pd

import wandb
from scripts.results_processing.preprocess import preprocess_df
from scripts.results_processing.reproduction.constants import (
    dataset_model_columns,
    optimization_metrics,
    run_columns,
    sweeped_columns,
)

%load_ext autoreload
%autoreload 2


In [2]:

columns_to_normalize = [
    "model",
    "dataset",
    "transforms",
    "optimizer",
    "callbacks",
]
def normalize_columns(df, columns_to_normalize):
    # Gather the new DataFrames to be concatenated
    flattened_dfs = []

    for col in columns_to_normalize:
        df[col] = df[col].apply(lambda x: str(x).replace("nan", "None"))
        df[col] = df[col].apply(literal_eval)

        # Flatten
        flat = pd.json_normalize(df[col])

        # Rename
        flat.columns = [f"{col}.{c}" for c in flat.columns]
        flattened_dfs.append(flat)

    # Drop all nested columns in one shot
    df = df.drop(columns=columns_to_normalize)

    # Concatenate once at the end
    return pd.concat([df] + flattened_dfs, axis=1)


def normalize_df(df, columns_to_normalize):
    # Config columns to normalize
    df = normalize_columns(df, columns_to_normalize)

    return df

def fetch():
    user = "telyatnikov_sap"
    project = "main_exp_GPSE_ogbg-molhiv"
    api = wandb.Api(overrides={"base_url": "https://api.wandb.ai"}, timeout=40)
    runs = api.runs(f"{user}/{project}")
    summary_list, config_list, name_list = [], [], []
    for run in runs:
        # .summary contains the output keys/values for metrics like accuracy.
        #  We call ._json_dict to omit large files
        summary_list.append(run.summary._json_dict)

        # .config contains the hyperparameters.
        #  We remove special values that start with _.
        config_list.append(
            {
                k: v
                for k, v in run.config.items()
                if not k.startswith("_")
            }
        )

        # .name is the human-readable name of the run.
        name_list.append(run.name)

    runs_df = pd.DataFrame(
        {
            "summary": summary_list,
            "config": config_list,
            "name": name_list,
        }
    )
    # Merge the dicts in a vectorized way:
    merged_dicts = [
        {**s, **c}
        for s, c in zip(runs_df["summary"], runs_df["config"], strict=False)
    ]

    # Now expand them into a DataFrame:
    df_merged = pd.DataFrame.from_records(merged_dicts)
    return df_merged

def main():
    df = fetch()
    df = normalize_df(df, columns_to_normalize)
    return df

In [3]:
def gen_scores(df):
    # Get unique datasets
    datasets = list(df["dataset.loader.parameters.data_name"].unique())
    # Get unique models
    models = list(df["model.model_name"].unique())

    collect_subsets = defaultdict(dict)
    # Got over each dataset and model and find the best result
    for dataset in datasets:
        for model in models:
            # Get the subset of the DataFrame for the current dataset and model
            subset = df.loc[
                (df["dataset.loader.parameters.data_name"] == dataset)
            ]

            optim_metric = optimization_metrics[dataset]["optim_metric"]
            eval_metric = optimization_metrics[dataset]["eval_metric"]
            direction = optimization_metrics[dataset]["direction"]

            # Keep metrics that matters for dataset
            performance_columns = optimization_metrics[dataset][
                "performance_columns"
            ]
            subset = subset[
                dataset_model_columns
                + sweeped_columns
                + performance_columns
                + run_columns
            ]
            aggregated = subset.groupby(
                sweeped_columns + ["model.model_name", "model.model_domain"],
                dropna=False,
            ).agg(
                {col: ["mean", "std", "count"] for col in performance_columns},
            )

            # aggregated = subset.groupby(sweeped_columns, dropna=False).count()

            n_count = 5 if "MANTRA" not in dataset else 4
            # Go from MultiIndex to Index
            aggregated = aggregated.reset_index()
            print(f"Dataset: {dataset}, Model: {model}")
            print(aggregated[(eval_metric, "count")].unique())
            # print(aggregated['dataset.split_params.data_seed'].unique())
            print(
                (aggregated[(eval_metric, "count")] >= n_count).sum()
                / len(aggregated)
                * 100
            )
            aggregated = aggregated[aggregated[(eval_metric, "count")] >= n_count]
            # print(len(aggregated[aggregated['seed'] > 4]))
            # aggregated = aggregated.sort_values(
            #     by=(optim_metric, "mean"), ascending=(direction == "min")
            # )

            # Git percent in case of classification
            if "test/accuracy" in performance_columns:
                # Go over all the performance columns and multiply by 100
                for col in performance_columns:
                    aggregated[(col, "mean")] *= 100
                    aggregated[(col, "std")] *= 100

                # Round performance columns values up to 2 decimal points
                for col in performance_columns:
                    aggregated[(col, "mean")] = aggregated[
                        (col, "mean")
                    ].round(4)
                    aggregated[(col, "std")] = aggregated[(col, "std")].round(
                        4
                    )

            else:
                # Round all values up to 4 decimal points
                # Round performance columns values up to 4 decimal points
                for col in performance_columns:
                    aggregated[(col, "mean")] = aggregated[
                        (col, "mean")
                    ].round(4)
                    aggregated[(col, "std")] = aggregated[(col, "std")].round(
                        4
                    )

            collect_subsets[dataset] = aggregated
    return collect_subsets


In [4]:
df = main()

In [5]:
df = preprocess_df(df, split_mantra=False)

In [6]:
collected_subsets = gen_scores(df)

Dataset: ogbg-molhiv, Model: hopse_g
[5 3 1 0 2 4]
88.13559322033898


In [21]:
collected_subsets["ogbg-molhiv"].sort_values(by=("val/auroc", "mean"), ascending=False)[["val/auroc", "test/auroc", "model.backbone.n_layers", "dataset.dataloader_params.batch_size", "optimizer.parameters.lr", "optimizer.parameters.weight_decay", "transforms.sann_encoding.pretrain_model", "transforms.sann_encoding.neighborhoods"]].head(60)

val/auroc               test/auroc               model.backbone.n_layers  \
         mean     std count       mean     std count                           
212   78.5561  1.5290     5    73.9272  3.0664     5                       1   
208   77.5297  1.1666     5    74.1542  2.7520     5                       1   
136   77.3994  2.2687     5    71.6806  2.7016     5                       2   
192   77.3197  3.5181     5    73.6264  1.6072     5                       2   
52    77.2984  2.9981     5    73.7953  2.5942     5                       1   
85    77.0527  2.2222     5    76.6055  1.0047     5                       1   
176   77.0196  3.4239     5    68.8513  2.4877     5                       1   
200   76.9960  2.0547     5    72.2979  1.7728     5                       2   
93    76.9950  3.5996     5    74.1606  0.7509     5                       1   
201   76.9879  2.4747     5    72.9734  2.2041     5                       2   
160   76.8369  1.9672     5    71.6078  3.1616     5                       2   
193   76.8190  3.2380     5    73.8707  2.1423     5                       2   
225   76.5028  1.5825     5    71.4217  2.1906     5                       2   
184   76.4060  1.8607     5    72.7869  1.7820     5                       1   
188   76.3814  1.9266     5    71.8260  1.5975     5                       2   
113   76.3739  2.3391     5    72.9503  2.2242     5                       1   
100   76.3709  3.3666     5    76.3001  1.6483     5                       2   
69    76.3420  2.2177     5    75.8339  1.8065     5                       2   
112   76.3289  2.7321     5    75.5313  2.7706     5                       1   
213   76.3028  2.4808     5    71.0903  1.9317     5                       1   
217   76.2406  1.1065     5    73.8724  1.3896     5                       1   
60    76.2215  0.9309     5    74.6143  0.7607     5                       1   
137   76.1917  2.2366     5    72.9882  3.2054     5                       2   
68    76.0454  3.0126     5    75.1053  1.3049     5                       2   
128   75.7603  2.4491     5    73.5049  3.4314     5                       2   
224   75.6392  2.2361     5    72.3209  1.5363     5                       2   
129   75.5753  2.5416     5    73.0540  1.5471     5                       2   
173   75.5103  2.4898     5    75.6231  1.2421     5                       1   
132   75.4826  4.6283     5    74.0012  1.4847     5                       2   
181   75.4549  1.0984     5    72.6732  2.7848     5                       1   
133   75.4332  3.1173     5    75.8240  1.8620     5                       2   
197   75.3941  1.3125     5    75.6026  1.2632     5                       2   
125   75.3398  1.4526     5    74.3069  1.9030     5                       2   
53    75.2930  1.7470     5    75.1033  2.1067     5                       1   
216   75.2912  2.3905     5    72.2357  2.2977     5                       1   
101   75.2693  3.1787     5    77.0170  0.9683     5                       2   
145   75.1108  2.5476     5    72.5107  2.6433     5                       1   
148   75.0590  2.2253     5    73.7980  1.1175     5                       1   
205   75.0294  3.2375     5    74.0993  2.0264     5                       1   
144   74.9649  3.5578     5    75.1185  2.1205     5                       1   
177   74.9473  2.3704     5    75.9956  1.7950     5                       1   
149   74.9093  3.4395     5    71.1836  3.0180     5                       1   
84    74.7982  4.9827     5    75.2295  2.3238     5                       1   
152   74.7941  1.1519     5    73.0435  1.9040     5                       1   
28    74.7436  1.6225     5    74.5479  1.6675     5                       1   
117   74.5759  3.1623     5    74.0380  1.9048     5                       1   
92    74.5442  3.7896     5    73.0011  0.7585     5                       1   
196   74.5042  2.0714     5    73.3908  2.8497     5                       2   
153 